In [1]:
using Pkg; Pkg.activate("../../julia_high_performance")

  Activating new project at `~/Documents/julia_high_performance`


# Third lecture: Julia

## GPU Computing

Originally designed for fast graphics calculations, they have found use in accelerating many kinds of numerical code. The defining characteristic of these processors has been their ability to run many threads (in the order of hundreds or thousands) in parallel, which then allows significant speedups in algorithms that can take advantage of this facility. Therefore, the general-purpose Graphics Processing Unit (__GPU__) has turned out to be one of the best ways of running fast parallel computations.  

Unlike most other languages, using the GPU from Julia is not just about calling out to pre-built C libraries: we are able to compile Julia code directly to the GPU.

In particular, we will discuss the following topics:  
- Basics of GPU architecture
- Getting started with GPUs
- CuArrays: Arrays on the GPU
- Writing kernels for GPUs
- How to benchmark, optimize, profile kernels

## Basics of GPGPU architecture  
A typical GPU architecture consists of:
- __Main global memory__  
    - Medium size ( 16-40 GB)  
    - Very high bandwidth ( 800-1200 GB/s)  
- __Streaming multiprocessors (SM)__  
    - Grouping independent cores and control units  
    - Number of SM depends on GPU architecture  

Each SM is organized in __blocks__ which are organized in __grids__.

Each SM unit has:  
- Many cores ( > 100 cores)  
- Lots of registers ( 32K-64K)  
- Instruction scheduler dispatchers  
- Shared memory with fast access to data  
- Several caches  

SM are composed of 4 independent blocks. Each block supports  
• M1 warps x 2 dispatchers  
• 16FP32 + 16INT32 ALU units  
• separate FP32 and INT32 cores, allowing simultaneous execution of FP32 and INT32 operations at full throughput  
• 8 FP64 ALU units  
• 2 Tensor Core units (HW matmul)  
• 8 Load/Store units  
• 4 SFU units  
• 32768 32bits registers  

Each block accesses:  
• 128KB for L1/shared memory  
• 4 texture units  

A full GA100 GPU unit contains 8 Compute Graphic Clusters (CGC) with 16 SM each, total 128 SMs:  
• 8192 FP32 cores  
• 8192 INT32 cores  
• 40MB L2 cache  

High Bandwidth Memory:  
• 40GB HBM2  
• 732 GB/s bandwidth

NVLink technology:  
• 600GB/s bandwidth to host data transfers  
• 24X respect PCIe Gen3 16x  

## CPUs vs GPUs
In CPUs the majority of silicon is dedicated to several ALUs and large caches. Moreover the memory hierarchy is organised in order to have very large DRAM and very fast and large cache memories (Registers, L1, L2, L3). Modern CPUs often have multiple cores on chip: each core has registers and ALU units to run a thread independently from other cores. When there are more threads on the fly than available cores, the operating system can make a __context switch__: the running thread is freezed and all information about its status is saved and put back for later restore. Then a new thread take the hardware resources, load its previous status and restart its flow until a new context switch will take place. This mechanism is optimized to __minimize latencies__. From this point of view CPUs tend to have tens of software threads at a time and perform task parallelism.

In GPUs the majority of silicon is dedicated to thousands of ALUs and each have its own control units and registers. The memory hierarchy is organised with high bandwidth DRAM and several cache memories whose storage capability is limited with respect to CPU ones. GPU scheduler organizes the work to do in SMs by grouping 32 threads in __warps__.  32 warps are called __thread blocks__ and are collected to fit __grid size__. GPU architecture __hides latencies__ with computation from thousands of other thread warps and tend to perform more a data parallelism. Several warps are mapped to an SM, and a SM instantaneously switches between the warps of a block. So each warp needs separate registers for each of its threads. Every block in a grid contains the same number of threads.


__CPUs pros__:  
- Large main memory ~ 1TB+  
- Faster clock speed ~ 4 GHz  
- Latency optimised via large caches  
- Small number of threads can run very quickly ~ Equal to number of cores  

__CPUs cons__:  
- Low memory bandiwdth (~200 Gb/s)  
- Every cache miss is a big cost in performances  
- Low ratio performance/power consumed   

__GPUs pros__:  
- High bandwidth main memory ~ 1TB/s+  
- Significantly more computing resources than CPUs  
- High throughput
- High ratio performance/power consumed  

__GPUs cons__:  
- Relatively low memory capacity (~80 Gb per GPU) 
- Low per-thread performance: 4 times slower clock speed than CPU cores  
- Depending on model, possible low transfer speeds  
- May need complete rewriting of applications



## GPGPU programming model
When talking about GPUs often we refer to __SIMT__ model (Single Instruction Multiple Threads). SIMT and SIMD both approach parallelism through broadcasting the same instruction to multiple execution units. This way, you replicate the execution units, but they all share the same fetch/decode hardware. In NVIDIA's model, there are 3 key features that SIMD doesn't have:

- Single instruction, multiple register sets
- Single instruction, multiple addresses
- Single instruction, multiple flow paths  


So, in NVIDIA's terminology CPU is referred as __Host__ while GPU is referred as __Device__. 



We have a host-centric model with one host and multiple devices of the same type. Devices are connected to host CPU via interconnect, such as PCIe or NVLink. Host and device have separate memory spaces.  
A function which runs on the device is called __Kernel__. Kernels are executed as a set of threads that can run concurrently and each thread is mapped to a single core on the GPU. Kernels are executed by using [CUDA](https://developer.nvidia.com/about-cuda). CUDA provides a coding paradigm that extends languages like C, C++, Python, and Fortran, to be capable of running accelerated, massively parallelized code. Unfortunately programming in CUDA is not always easy, especially for large and complicated codes. 

Data must be moved from host to device memory in order to be processed by a CUDA kernel. When data is processed, and no more is needed on the GPU, it is transferred back to host. Connections to GPU are relatively slow so it is crucial to minimize data transfers because they can become a significant bottleneck. Programmers choose the number of threads to run and each thread can act on a different data element independently. Threads belonging to the same block or team can cooperate together exchanging data through a shared memory cache area.


The GPU’s threads need to be saturated otherwise the result will be slower than CPU. GPU programmers need to write codes that exploit the maximum __occupancy__ available for the specific device. For maximum performance, execution on the CPU should continue as much as possible while GPU is being used.

## Getting started with GPUs in Julia  
Programming GPUs needs special considerations in terms of the design and architecture of your programs. You need to write computations as vectorized operations, and be careful in how (and how often) data is copied from the CPU and GPU.

In [ ]:
# using Pkg
# Pkg.instantiate()

In [ ]:
# Pkg.add("CUDA")

In [ ]:
# Pkg.add("GPUArrays")

In [2]:
using CUDA, GPUArrays, BenchmarkTools, InteractiveUtils, Test

In [9]:
;env nvidia-smi

Tue Feb 27 22:36:19 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 470.42.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  On   | 00000000:3B:00.0 Off |                    0 |
| N/A   29C    P0    26W / 250W |     26MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
CUDA.name(CuDevice(0))

"Tesla V100S-PCIE-32GB"

In [11]:
CUDA.versioninfo();

CUDA runtime 11.5, local installation
CUDA driver 11.4
NVIDIA driver 470.42.1

CUDA libraries: 
- CUBLAS: 11.7.3
- CURAND: 10.2.6
- CUFFT: 10.6.0
- CUSOLVER: 11.2.1
- CUSPARSE: 11.7.0
- CUPTI: 16.0.0
- NVML: 11.0.0+470.42.1

Julia packages: 
- CUDA.jl: 5.2.0
- CUDA_Driver_jll: 0.7.0+1
- CUDA_Runtime_jll: 0.11.1+0
- CUDA_Runtime_Discovery: 0.2.3

Toolchain:
- Julia: 1.8.4
- LLVM: 13.0.1

Preferences:
- CUDA_Runtime_jll.version: 11.5
- CUDA_Runtime_jll.local: true

1 device:
  0: Tesla V100S-PCIE-32GB (sm_70, 31.722 GiB / 31.749 GiB available)


## CuArrays  
The CuArrays package is probably the most significant part of the GPU ecosystem in Julia: it provides an array type for storing data on the GPU. Since GPU code needs to be vectorized, an array is the primary data type that needs to be used in all cases.  
Once created, GPU kernels may be called on it and array operations are directly performed on GPU. The package includes kernels for basic mathematical operators, which are typically called via dot-broadcast. As with all broadcasts in Julia, the operations are fused, which means that on the GPU, a single fused kernel is run on the array, even though multiple operations exist on the source. In other words, the power, subtraction, multiplication, addition, and squaring in the following expression is all combined into a single block of processing:

In [12]:
a = CuArray([1f0, 2f0, 3f0])

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 2.0
 3.0

In [13]:
b = a.^2 .- a.*2 .+ sqrt.(a)

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 1.4142135
 4.732051

`1f0` is Julia shorthand for creating a 32-bit float, or a Float32. By default, a floating point number in Julia (such as 1.0) is 64-bit, also known as Float64.  
There are a whole range of array operations to process your data with `CuArray`. This includes several higher-order operations that take other code as arguments, such as `map`, `reduce` or `broadcast`. With these, it is possible to perform kernel-like operations without actually writing your own GPU kernels.

In [14]:
map(b) do x 
    x^2
end 

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
  0.0
  1.9999999
 22.392305

In [15]:
a = CUDA.zeros(1024)
b = CUDA.ones(1024)
a.^2 .+ sin.(b)

1024-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 ⋮
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471
 0.841471

In [16]:
a = CUDA.ones(2,3)

2×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  1.0  1.0
 1.0  1.0  1.0

In [17]:
reduce(+, a)

6.0f0

In [18]:
mapreduce(sin, *, a; dims=2)

2×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.59582335
 0.59582335

In [19]:
b = CUDA.zeros(1)
Base.mapreducedim!(identity, +, b, a)  #shrinking dimensions!

1×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 6.0

### Random numbers
Base's convenience functions for generating random numbers are available in the CUDA module as well. The __`CUDA.rand`__ function generates a set of `n` random numbers on the GPU (this creates a `CuArray`):

In [20]:
CUDA.rand(2)

2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.16409281
 0.7293283

In [21]:
CUDA.randn(Float64, 2, 1)

2×1 CuArray{Float64, 2, CUDA.Mem.DeviceBuffer}:
 -0.5618517152237089
 -0.943638769392992

Behind the scenes, these random numbers come from two different generators: one backed by `CURAND`, another by kernels defined in `CUDA.jl`. Operations on these generators are implemented using methods from the `Random` standard library:

In [23]:
using Random

In [24]:
a = Random.rand(CURAND.default_rng(), Float32, 1)

1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.9250437

In [25]:
a = Random.rand!(CUDA.default_rng(), a)

1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.22773993

In [115]:
r = CUDA.rand(Float32, 1000000);
s = Random.rand(Float32, 1000000);

#Then try this example with a much bigger or much smaller number of elements!

In [58]:
@btime r.^2

  8.648 μs (47 allocations: 1.92 KiB)


1000000-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.6802501
 0.12181059
 0.9125508
 0.06877528
 0.42336515
 0.5301062
 0.09092845
 0.53041947
 0.17369501
 0.1992623
 0.20332655
 0.16388033
 0.2285276
 ⋮
 0.0007550328
 0.50100803
 0.29893264
 0.52745163
 0.25905254
 0.0003614126
 0.9108409
 0.8943569
 0.08393471
 0.9397194
 0.849757
 0.07782332

In [59]:
@btime s.^2

  479.184 μs (6 allocations: 3.81 MiB)


1000000-element Vector{Float32}:
 0.25592494
 0.32113475
 0.25497317
 0.15508123
 0.0027577844
 0.0033740501
 0.008886292
 0.277484
 0.011235243
 0.6571071
 0.47538236
 0.32533577
 0.2900139
 ⋮
 0.6024891
 0.044956062
 0.2945667
 0.2057178
 0.036944095
 0.01361733
 0.742561
 0.3237882
 0.40307134
 0.5992009
 0.9984385
 0.0081792595

In [60]:
r = CUDA.rand(Float64, 1000000);
s = Random.rand(Float64, 1000000);

In [61]:
@btime r.^2

  8.828 μs (47 allocations: 1.92 KiB)


1000000-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 0.17082575871594843
 0.036461194790803224
 0.02836488302781119
 0.3409969976961764
 0.023517211867152617
 0.4715446418145448
 0.49290449924629626
 0.0006513068355554904
 0.7613496132873004
 0.3917316693792084
 0.6627414328299668
 0.08149161283832242
 0.7849824508651738
 ⋮
 0.9452266085831253
 0.710898265634248
 0.49805315169536757
 0.021140805767054256
 0.1257464986592408
 0.5981379005845393
 0.7140101438332606
 0.6275737466620733
 0.5536316523799842
 0.6067524485492398
 0.37593449127808365
 0.5131176218024278

In [62]:
@btime s.^2

  1.003 ms (6 allocations: 7.63 MiB)


1000000-element Vector{Float64}:
 0.006305856432214003
 0.3785656824705665
 0.12887053464219048
 0.2880990290985759
 0.7193931024384013
 0.20877629536472733
 0.4979505623090851
 0.00991309143908488
 0.2644411695314684
 0.0010063035149077816
 0.18698343152805036
 0.05613844437671575
 0.02928534087624466
 ⋮
 0.48686749603078855
 0.004873085641523639
 0.3139202553364028
 0.02219962307399153
 0.11274534790726443
 0.02163676980690811
 0.3129744666462199
 0.8775155425120534
 0.9633631733712182
 0.31377344000135277
 0.1609600624489402
 0.10177903305057986

Using `Float32` made the code run faster. If your algorithm can work with the precision of 32 bits of floating point, it can be hugely advantageous to use that instead of `Float64`.  

It is also possible to initialize a `CuArray` by calling other methods from `Base.Array` in it (and so there is full API compatibility):  

In [64]:
x = CuArray(rand(Float32, 10))

10-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.8859392
 0.28099722
 0.9523352
 0.7116148
 0.9003665
 0.11976361
 0.8251001
 0.9207761
 0.6977102
 0.58192617

### Memory allocation and managing
Data transfer happens explicitly at the creation of a `CuArray`. `CuArray` in fact contains a device pointer, so it is only the representation of the device memory and a copy on GPU is created.

In [65]:
a = CuArray([4., 5., 6.])
b = CuArray([4., 5., 6.])

3-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 4.0
 5.0
 6.0

In [66]:
a .+ 2*b

3-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 12.0
 15.0
 18.0

In [67]:
a = CuArray{Int}(undef, 1024)

# essential memory operations, like copying, filling, reshaping, ...
b = copy(a)
fill!(b, 0)
@test b == CUDA.zeros(Int, 1024)

# automatic memory management
a = nothing

In many cases, you might not want to convert your input data to a dense `CuArray`. __`cu()`__ constructor preserve array type (or wrapper array type) on the GPU and only upload the contained data. The `cu` function is opinionated and converts input most floating-point scalars to `Float32`. This is often a good call, as we already know that `Float64` and many other scalar types perform badly on the GPU.

In [68]:
# allocate to CPU memory
len = Int64(1e8);
h_a, h_b = zeros(Float32, len), rand(Float32, len)

(Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.20377517, 0.30827266, 0.12709904, 0.013810575, 0.04688996, 0.27041447, 0.79867435, 0.2037813, 0.39135385, 0.08346909  …  0.34831637, 0.7795897, 0.9393169, 0.44849032, 0.03455472, 0.66908485, 0.7657099, 0.5878089, 0.63311654, 0.8281715])

In [69]:
# transfer to GPU memory
d_a, d_b = cu(h_a), cu(h_b)

(Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.20377517, 0.30827266, 0.12709904, 0.013810575, 0.04688996, 0.27041447, 0.79867435, 0.2037813, 0.39135385, 0.08346909  …  0.34831637, 0.7795897, 0.9393169, 0.44849032, 0.03455472, 0.66908485, 0.7657099, 0.5878089, 0.63311654, 0.8281715])

The `CuArray` constructor and the `cu` function default to allocating device memory, which can be accessed only from the GPU. It is also possible to allocate __unified memory__, which is accessible from both the CPU and GPU with the driver taking care of data movement:

In [70]:
cpu = [1,2]

2-element Vector{Int64}:
 1
 2

In [71]:
gpu = CuVector{Int,Mem.Unified}(cpu)

2-element CuArray{Int64, 1, CUDA.Mem.UnifiedBuffer}:
 1
 2

In [72]:
gpu = cu(cpu; unified=true)

2-element CuArray{Int64, 1, CUDA.Mem.UnifiedBuffer}:
 1
 2

Using unified memory has several advantages: it is possible to allocate more memory than the GPU has available, and the memory can be accessed efficiently from the CPU, either directly or by wrapping the `CuArray` using an Array.  
This may make it significantly easier to port code to the GPU, as you can incrementally port parts of your application without having to worry about executing CPU code, or triggering an `AbstractArray` fallback. It may come at a cost however, as unified memory needs to be paged in and out of the GPU memory, and cannot be allocated asynchronously. To alleviate this cost, `CUDA.jl` automatically prefetches unified memory when passing it to a kernel.

### Linear algebra and solver
CUDA's linear algebra functionality from the __`CUBLAS`__ library is exposed by implementing methods in the LinearAlgebra standard library:

In [73]:
# enable logging to demonstrate a CUBLAS kernel is used
CUBLAS.cublasLoggerConfigure(1, 0, 1, C_NULL)

In [74]:
CUDA.rand(2,2) * CUDA.rand(2,2)

I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=0
i!  value: type=int; val=POINTER (IN HEX:0x0x151769fc1140)
i! Time: 2024-02-27T22:58:03 elapsed from start 22.483333 minutes or 1349.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=1
i!  value: type=int; val=POINTER (IN HEX:0x0x151769fc1150)
i! Time: 2024-02-27T22:58:03 elapsed from start 22.483333 minutes or 1349.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=2
i!  value: type=int; val=POINTER (I

2×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.201579  0.0521619
 0.218757  0.0802236

Operations that exist in `CUBLAS`, but are not (yet) covered by high-level constructs in the `LinearAlgebra` standard library, can be accessed directly from the `CUBLAS` submodule. 

In [75]:
x = CUDA.rand(2)

2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.4951467
 0.6098753

In [76]:
y = CUDA.rand(2)

2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.710232
 0.08478106

In [77]:
CUBLAS.dot(2, x, y)

I! cuBLAS (v11.5) function cublasStatus_t cublasSdot_v2(cublasHandle_t, int, const float*, int, const float*, int, float*) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x186eb1d0)
i!  n: type=int; val=2
i!  x: type=float; val=POINTER (IN HEX:0x0x302002000)
i!  incx: type=int; val=1
i!  y: type=float; val=POINTER (IN HEX:0x0x302004200)
i!  incy: type=int; val=1
i!  result: type=float; val=POINTER (IN HEX:0x0x151760e342c0)
i! Time: 2024-02-27T22:58:43 elapsed from start 23.150000 minutes or 1389.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x0x186eb1d0); StreamId=POINTER (IN HEX:0x0x6b3bd40); MathMode=CUBLAS_DEFAULT_MATH | CUBLAS_MATH_DISALLOW_REDUCED_PRECISION_REDUCTION
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)


0.4033749f0

`LAPACK`-like functionality as found in the __`CUSOLVER`__ library can be accessed through methods in the `LinearAlgebra` standard library too:

In [78]:
using LinearAlgebra

In [79]:
a = CUDA.rand(2,2)

2×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.984164  0.585097
 0.789472  0.974074

In [80]:
a = a * a'

I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=0
i!  value: type=int; val=POINTER (IN HEX:0x0x151760f72020)
i! Time: 2024-02-27T22:59:05 elapsed from start 23.516667 minutes or 1411.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=1
i!  value: type=int; val=POINTER (IN HEX:0x0x151760f72030)
i! Time: 2024-02-27T22:59:05 elapsed from start 23.516667 minutes or 1411.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasGetProperty(libraryPropertyType, int*) called:
i!  type: type=SOME TYPE; val=2
i!  value: type=int; val=POINTER (I

2×2 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.31092  1.3469
 1.3469   1.57209

In [81]:
cholesky(a)

I! cuBLAS (v11.5) function cublasStatus_t cublasCreate_v2(cublasContext**) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x27eeb6f0)
i! Time: 2024-02-27T22:59:15 elapsed from start 23.683333 minutes or 1421.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasCreate_v2(cublasContext**) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x27eeb738)
i! Time: 2024-02-27T22:59:15 elapsed from start 23.683333 minutes or 1421.000000 seconds
i!Process=42027; Thread=23196185539520; GPU=0; Handle=POINTER (IN HEX:0x(nil))
i! COMPILED WITH: GNU GCC/G++ / 6.3.1 20170216 (Red Hat 6.3.1-3)
I! cuBLAS (v11.5) function cublasStatus_t cublasCreate_v2(cublasContext**) called:
i!  handle: type=cublasHandle_t; val=POINTER (IN HEX:0x0x27eeb740)
i! Time: 2024-02-27T22:59:16 elapsed from start 23.700000 minutes or 1422.000000 sec

Cholesky{Float32, CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}}
U factor:
2×2 UpperTriangular{Float32, CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}}:
 1.14495  1.17638
  ⋅       0.433844

## Writing a kernel 
When array operations are not flexible enough, you can write your own GPU kernels in Julia. Writing kernels in Julia is very similar to writing kernels in CUDA C/C++. It should be possible to learn CUDA programming from existing CUDA C/C++ resources, and apply that knowledge to programming in Julia using `CUDA.jl`.

The first naive approach is to run a function with `@cuda`. This automatically (re)compiles the kernel function and launches it on the current GPU.

In [82]:
function my_kernel()
    return
end

@cuda my_kernel()

CUDA.HostKernel for my_kernel()

GPU kernels __cannot return values__, and should always return or return nothing on all code paths. To communicate values from a kernel, you can use a `CuArray`:

In [83]:
function my_kernel(a)
    a[1] = 42
    return
end

my_kernel (generic function with 2 methods)

In [84]:
a = CuArray{Int}(undef, 1);
@cuda my_kernel(a);
a

1-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 42

GPU operations always execute __asynchronously__, so we need to wait for the GPU to finish before we can access the result. This is not needed when using `CuArrays`, as they automatically synchronize on access.  

Simply using `@cuda` only launches a single thread, which is not very useful. To launch more threads, use the `threads` and `blocks` keyword arguments to `@cuda`, while using indexing intrinsics in the kernel to differentiate the computation for each thread:

In [85]:
function my_kernel(a)
    i = threadIdx().x
    a[i] = 42
    return
end

my_kernel (generic function with 2 methods)

In [86]:
a = CuArray{Int}(undef, 5);
@cuda threads=length(a) my_kernel(a);

In [87]:
a

5-element CuArray{Int64, 1, CUDA.Mem.DeviceBuffer}:
 42
 42
 42
 42
 42

### Standard output

In [88]:
# this is going to be executed on GPU
function hello()
    @cuprintf("Hello, world!\n")
    return 
end

hello (generic function with 1 method)

In [89]:
@cuda hello()

CUDA.HostKernel for hello()

Hello, world!


In [90]:
@cuda threads = 4 hello()

CUDA.HostKernel for hello()

Hello, world!
Hello, world!
Hello, world!
Hello, world!


In [91]:
function hello!()
    @cuprintf("Hello, world! from thread %u\n", threadIdx().x)
    return 
end
@cuda hello!();

Hello, world! from thread 1


In [100]:
println("Something on the way....")
@cuda threads = 1024 hello!() 
println("mmmmm....")

Something on the way....
mmmmm....
Hello, world! from thread 897
Hello, world! from thread 898
Hello, world! from thread 899
Hello, world! from thread 900
Hello, world! from thread 901
Hello, world! from thread 902
Hello, world! from thread 903
Hello, world! from thread 904
Hello, world! from thread 905
Hello, world! from thread 906
Hello, world! from thread 907
Hello, world! from thread 908
Hello, world! from thread 909
Hello, world! from thread 910
Hello, world! from thread 911
Hello, world! from thread 912
Hello, world! from thread 913
Hello, world! from thread 914
Hello, world! from thread 915
Hello, world! from thread 916
Hello, world! from thread 917
Hello, world! from thread 918
Hello, world! from thread 919
Hello, world! from thread 920
Hello, world! from thread 921
Hello, world! from thread 922
Hello, world! from thread 923
Hello, world! from thread 924
Hello, world! from thread 925
Hello, world! from thread 926
Hello, world! from thread 927
Hello, world! from thread 928
Hello

Launching kernels is asynchronous! The CPU code will continue to execute without waiting for the kernel launch to complete. In order to make host wait for the device to complete its work, a `syncronize()` is required:

In [101]:
println("Something on the way....")
@cuda threads = 1024 hello!()
synchronize();
println("mmmmm....")

Something on the way....
Hello, world! from thread 961
Hello, world! from thread 962
Hello, world! from thread 963
Hello, world! from thread 964
Hello, world! from thread 965
Hello, world! from thread 966
Hello, world! from thread 967
Hello, world! from thread 968
Hello, world! from thread 969
Hello, world! from thread 970
Hello, world! from thread 971
Hello, world! from thread 972
Hello, world! from thread 973
Hello, world! from thread 974
Hello, world! from thread 975
Hello, world! from thread 976
Hello, world! from thread 977
Hello, world! from thread 978
Hello, world! from thread 979
Hello, world! from thread 980
Hello, world! from thread 981
Hello, world! from thread 982
Hello, world! from thread 983
Hello, world! from thread 984
Hello, world! from thread 985
Hello, world! from thread 986
Hello, world! from thread 987
Hello, world! from thread 988
Hello, world! from thread 989
Hello, world! from thread 990
Hello, world! from thread 991
Hello, world! from thread 992
Hello, world! f

Or sometimes it is better to use `CUDA.@sync` macro:  

In [102]:
println("Something on the way....")
CUDA.@sync @cuda threads = 1024 hello!()
println("mmmmm....")

Something on the way....
Hello, world! from thread 801
Hello, world! from thread 802
Hello, world! from thread 803
Hello, world! from thread 804
Hello, world! from thread 805
Hello, world! from thread 806
Hello, world! from thread 807
Hello, world! from thread 808
Hello, world! from thread 809
Hello, world! from thread 810
Hello, world! from thread 811
Hello, world! from thread 812
Hello, world! from thread 813
Hello, world! from thread 814
Hello, world! from thread 815
Hello, world! from thread 816
Hello, world! from thread 817
Hello, world! from thread 818
Hello, world! from thread 819
Hello, world! from thread 820
Hello, world! from thread 821
Hello, world! from thread 822
Hello, world! from thread 823
Hello, world! from thread 824
Hello, world! from thread 825
Hello, world! from thread 826
Hello, world! from thread 827
Hello, world! from thread 828
Hello, world! from thread 829
Hello, world! from thread 830
Hello, world! from thread 831
Hello, world! from thread 832
Hello, world! f

To simply show a value, which can be useful during debugging, use `@cushow`:

In [103]:
@cuda threads=2 (()->(@cushow threadIdx().x; return))()

CUDA.HostKernel for #3()

(threadIdx()).x = 1
(threadIdx()).x = 2


### Specifications on how Julia compiles kernels
`hello()` is a regular Julia function, even though it uses some GPU-specific invocations. We have run this function on the GPU via the `@cuda` macro which generates specialized code for compiling the kernel function to GPU assembly.  
NVIDIA uses a pseudo-assembly intermediate language known as Parallel Thread Execution (__PTX__). We generate PTX from Julia code using the LLVM PTX compiler backend. Once generated, the PTX is converted to binary code by a compiler built into the device driver.  
We can inspect the generated code that is actually run on the device with `@device_code_ptx` macro or the binary code itself with `@device_code_sass`:

In [104]:
@device_code_ptx @cuda hello()

// PTX CompilerJob of MethodInstance for hello() for sm_70

//
// Generated by LLVM NVPTX Back-End
//

.version 7.4
.target sm_70
.address_size 64

	// .globl	_Z5hello                // -- Begin function _Z5hello
.extern .func  (.param .b32 func_retval0) vprintf
(
	.param .b64 vprintf_param_0,
	.param .b64 vprintf_param_1
)
;
.global .align 1 .b8 __unnamed_1[15] = {72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 33, 10, 0};
                                        // @_Z5hello
.visible .entry _Z5hello(
	.param .align 8 .b8 _Z5hello_param_0[16]
)
{
	.reg .b32 	%r<3>;
	.reg .b64 	%rd<4>;

// %bb.0:                               // %conversion
	mov.u64 	%rd1, __unnamed_1;
	cvta.global.u64 	%rd2, %rd1;
	mov.u64 	%rd3, 0;
	{ // callseq 79, 0
	.reg .b32 temp_param_reg;
	.param .b64 param0;
	st.param.b64 	[param0+0], %rd2;
	.param .b64 param1;
	st.param.b64 	[param1+0], %rd3;
	.param .b32 retval0;
	call.uni (retval0), 
	vprintf, 
	(
	param0, 
	param1
	);
	ld.param.b32 	%r1, [retval0+0

In [105]:
@device_code_sass @cuda hello()

// PTX CompilerJob of MethodInstance for hello() for sm_70

	.headerflags	@"EF_CUDA_TEXMODE_UNIFIED EF_CUDA_64BIT_ADDRESS EF_CUDA_SM70 EF_CUDA_VIRTUAL_SM(EF_CUDA_SM70)"
	.elftype	@"ET_EXEC"


//--------------------- .text._Z5hello            --------------------------
	.section	.text._Z5hello,"ax",@progbits
	.sectioninfo	@"SHI_REGISTERS=24"
	.align	128
        .global         _Z5hello
        .type           _Z5hello,@function
        .size           _Z5hello,(.L_x_2 - _Z5hello)
        .other          _Z5hello,@"STO_CUDA_ENTRY STV_DEFAULT"
_Z5hello:

.text._Z5hello:
        IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;
   @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;
        MOV R4, 32@lo(__unnamed_1) ;
        CS2R R6, SRZ ;
        MOV R5, 32@hi(__unnamed_1) ;
        MOV R20, 32@lo((_Z5hello + .L_x_0@srel)) ;
        MOV R21, 32@hi((_Z5hello + .L_x_0@srel)) ;
        CALL.ABS.NOINC `(vprintf) ;

.L_x_0:
        EXIT ;

.L_x_1:
        BRA `(.L_x_1);
        NOP;
        NOP;
        NOP;
        NO

Together with the JIT compiler, this results in a very efficient kernel launch sequence, avoiding runtime overhead typically associated with dynamic type languages. Julia integrates its compiler as much as possible, since only in the final step machine code is fully specific to the GPU package. Calling the same kernel with differently typed arguments “just works”.  


## Benchmarking, profiling and optimizing
Benchmarking and profiling a GPU program is harder than doing the same for a program executing on the CPU. GPU operations typically execute asynchronously, and thus require appropriate synchronization when measuring their execution time. Furthermore, because the program executes on a different processor, it is much harder to know what is currently executing.  

### Benchmarking
To accurately measure execution time in the presence of asynchronously-executing GPU operations, `CUDA.jl` provides an `@elapsed` macro that, much like `Base.@elapsed`, measures the total execution time of a block of code on the GPU

In [106]:
a = CUDA.rand(1024,1024,1024);

In [107]:
Base.@elapsed sin.(a)  # WRONG!

0.44243194

In [108]:
CUDA.@elapsed sin.(a)

0.031900194f0

This is a low-level utility, and measures time by submitting events to the GPU and measuring the time between them. As such, if the GPU was not idle in the first place, you may not get the expected result. The macro is mainly useful if your application needs to know about the time it took to complete certain GPU operations.  

For more convenient time reporting, you can use the `CUDA.@time` macro which mimics `Base.@time` by printing execution times as well as memory allocation stats, while making sure the GPU is idle before starting the measurement, as well as waiting for all asynchronous operations to complete:

In [109]:
CUDA.@time sin.(a);

  0.033039 seconds (89 CPU allocations: 4.812 KiB) (1 GPU allocation: 4.000 GiB, 39.51% memmgmt time)


The `CUDA.@time` macro is more user-friendly and is a generally more useful tool when measuring the end-to-end performance characteristics of a GPU application.

For robust measurements however, it is advised to use the `BenchmarkTools.jl` package which goes to great lengths to perform accurate measurements. Due to the asynchronous nature of GPUs, you need to ensure the GPU is synchronized at the end of every sample, e.g. by calling `synchronize()` or, even better, wrapping your code in `CUDA.@sync`:

In [110]:
@benchmark CUDA.@sync sin.($a)

BenchmarkTools.Trial: 270 samples with 1 evaluation.
 Range (min … max):  16.659 ms … 33.243 ms  ┊ GC (min … max): 0.00% … 3.83%
 Time  (median):     17.337 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   18.572 ms ±  2.249 ms  ┊ GC (mean ± σ):  1.67% ± 2.52%

     ▁██▅                                ▅▄                    
  ▆▄▇████▄█▄▄▁▆▄▄▁▁▁▁▁▄▆▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁███▇▄▁▁▁▁▁▁▁▆▁▄▁▁▄▆▁▄ ▆
  16.7 ms      Histogram: log(frequency) by time      23.8 ms <

 Memory estimate: 4.75 KiB, allocs estimate: 85.

The allocations as reported by `BenchmarkTools` are CPU allocations. For the GPU allocation behavior you need to consult `CUDA.@time`.  

### Comparing host and device performances

In [111]:
len = Int64(1e8);
h_a, h_b = zeros(Float32, len), rand(Float32, len)

(Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.021223962, 0.32138503, 0.69496566, 0.1858226, 0.5341011, 0.9257922, 0.69186896, 0.035246193, 0.08353484, 0.13071734  …  0.92163754, 0.603091, 0.88655275, 0.23053414, 0.9161499, 0.050336063, 0.6844901, 0.8217529, 0.8797914, 0.49619055])

In [112]:
d_a, d_b = cu(h_a), cu(h_b)

(Float32[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float32[0.021223962, 0.32138503, 0.69496566, 0.1858226, 0.5341011, 0.9257922, 0.69186896, 0.035246193, 0.08353484, 0.13071734  …  0.92163754, 0.603091, 0.88655275, 0.23053414, 0.9161499, 0.050336063, 0.6844901, 0.8217529, 0.8797914, 0.49619055])

In [113]:
host_bench =  @btime map!($sin, $h_a, $h_b);

  749.997 ms (0 allocations: 0 bytes)


In [114]:
device_bench = @btime map!($sin, $d_a, $d_b);

  11.784 μs (73 allocations: 3.75 KiB)


In [116]:
# first benchmark
# Map transform arrays by applying f to each element. For multiple collection arguments, 
# apply f elementwise, and stop when any of them is exhausted
# map! is like map, but stores the result in destination

# Computation on the CPU
host_bench =  @belapsed map!($sin, $h_a, $h_b);

# Computation on the GPU
device_bench = @belapsed map!($sin, $d_a, $d_b);

# Performance
times = [host_bench, device_bench]
speedup = maximum(times) ./ times

2-element Vector{Float64}:
     1.0
 60119.14837625979

We could also test again the full API compatibility of `CuArray` with `Base.Array` methods:

In [117]:
for Typ in (CuArray, Array)
    len = 2^18
    a = Typ(fill(2.0f0, len))
    b = Typ(fill(1.0f0, len))
    c = similar(a)
    t = @elapsed begin
        for i in eachindex(a, b)
            c .= a .* b
            CUDA.@sync(c)
        end
    end

    if c isa CuArray
        println("GPU time: ", t)
    else
        println("CPU time: ", t)
    end
end

GPU time: 4.387809127
CPU time: 34.377108169


### Profling
We cannot use CPU utilities to profile GPU programs, as they will only paint a partial picture. Instead, `CUDA.jl` provides a `CUDA.@profile` macro that separately reports the time spent on the CPU, and the time spent on the GPU:

In [ ]:
a = CUDA.rand(Float32, 100)
CUDA.@profile sin.(a)

┌ Error: The integrated profiler is not supported on Julia <1.9, and will crash.
└ @ CUDA.Profile ~/.julia/packages/CUDA/htRwP/src/profile.jl:264


By default, `CUDA.@profile` will provide a summary of host and device activities. If you prefer a chronological view of the events, you can set the `trace` keyword argument:

In [ ]:
CUDA.@profile trace=true sin.(a)

If you want more details, or a graphical representation, external profilers are recommended. NVIDIA provides two tools for profiling CUDA applications: __NSight Systems__ and __NSight Compute__ for respectively timeline profiling and more detailed kernel analysis. Both tools are well-integrated with the Julia GPU packages, and make it possible to iteratively profile without having to restart Julia.

### Optimizing 
- __Minimize data transfer between the CPU and GPU__: you can do this by getting rid of unnecessary memory copies and batching many small transfers into larger ones;
- Identify __problematic kernel invocations__: you may be launching thousands of kernels which could be fused into a single call;  
- Find __stalls__, where the CPU isn't submitting work fast enough to keep the GPU busy;  
- __Memory optimizations__ are the most important area for performance. Hence optimizing memory accesses, e.g., avoiding needless global accesses (buffering in shared memory instead) and coalescing accesses can lead to big performance improvements;  
- Launching more threads on each streaming multiprocessor can be achieved by __lowering register pressure__ and __reducing shared memory usage__;  
- __Using Float32's instead of Float64's__ can provide significantly better performance;  
- Avoid using __control flow__ instructions such as `if` which cause branches;
- Increase the __arithmetic intensity__ in order for the GPU to be able to hide the latency of memory accesses;  
- __Inlining__ can reduce register usage and thus speed up kernels. To force inlining of all functions use `@cuda always_inline=true`;
- The number of threads that can be launched is partly determined by the number of registers a kernel uses. This is due to registers being shared between all threads on a multiprocessor. __Setting the maximum number of registers per thread__ will force less registers to be used which can increase thread count at the expense of having to spill registers into local memory, this may improve performance. To set the max registers to 32 use `@cuda max_registers=32`;  
- __Use `@fastmath`__ to use faster versions of common mathematical functions and for even faster square roots use `@cuda fastmath=true`;
- __Minimize runtime exceptions__: by default, indexing a CuDeviceArray will perform bounds checking, and throw an error when the index is out of bounds. This can be a costly operation, so make sure to use __`@inbounds`__ when you know the index is in bounds.

### Porting a multiplication function to GPU  
Consider the following function:  

In [3]:
function gpu_mult1!(a, b, c)
    for i = 1:length(c)
        @inbounds c[i] = a[i] * b[i]
    end
    return nothing
end

gpu_mult1! (generic function with 1 method)

In [10]:
N = Int64(2^20);
a_d, b_d = CUDA.fill(1.0f0, N), CUDA.fill(2.0f0, N) ;  # a vector stored on the GPU filled with 1.0 (Float32)
c_d = similar(a_d)

1048576-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
# first porting attempt
@btime @cuda threads=1024 gpu_mult1!(a_d, b_d, c_d);

  14.988 μs (26 allocations: 1.38 KiB)


In [12]:
@test all(Array(c_d) .== 2.0f0)

Test Passed

In [15]:
@belapsed gpu_mult1!(a_d, b_d, c_d)

29.037554386

All these timing macros from `BenchmarkTools` work well generally, but there are some GPU specific methods that may occasionally be preferable.  
`@sync` macro may be used to force the execution of asynchronous kernels. These are usually only executed when its results are required by downstream processes, but that obviously invalidates the performance measurement. The `@sync` macro mitigates this issue by forcing the computation to occur within the
timing loop.

In [16]:
function bench_gpu1!(a, b, c)
    CUDA.@sync begin
        @cuda threads=256 gpu_mult1!(a, b, c)
    end
end

bench_gpu1! (generic function with 1 method)

Note, however, that whenever you copy data back to the CPU, the code is forced to run, and `@sync` is not required when measuring. This copy can be implicit, for example, when displaying an output in the REPL. On the other hand, in these situations, you are also measuring the time to copy the data from the GPU to the CPU. So be cognizant of what exactly you are measuring.  

In [18]:
c_d = CUDA.zeros(Float32, N);
@btime bench_gpu1!(a_d, b_d, c_d)

  63.378 ms (70 allocations: 4.25 KiB)


CUDA.HostKernel for gpu_mult1!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

To speed up the kernel, we want to parallelize it, which means assigning different tasks to different threads. In order to facilitate the assignment of work, each CUDA thread gets access to variables that indicate its own unique identity, much as `Threads.threadid()` does for CPU threads. The CUDA analogs of `threadid` and `nthreads` are called `threadIdx` and `blockDim`.

In [23]:
function gpu_mult2!(a, b, c)
    index = threadIdx().x    # this example only requires linear indexing, so just use `x`
    stride = blockDim().x
    # @cuprintln("thread $index, block $stride")
    for i = index:stride:length(c)
        @inbounds c[i] = a[i] * b[i]
    end
    return nothing
end

gpu_mult2! (generic function with 1 method)

In [24]:
c_d = CUDA.zeros(Float32, N)
@cuda threads=1024 gpu_mult2!(a_d, b_d, c_d)

CUDA.HostKernel for gpu_mult2!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [ ]:
@test all(Array(c_d) .== 2.0f0)

In [25]:
function bench_gpu2!(a, b, c)
    CUDA.@sync begin
        @cuda threads=256 gpu_mult2!(a, b, c)
    end
end

bench_gpu2! (generic function with 1 method)

In [26]:
c_d = CUDA.zeros(Float32, N)
@btime bench_gpu2!(a_d, b_d, c_d);

  1.541 ms (69 allocations: 4.02 KiB)


## Interlude: Mapping threads on the GPU

__ACHTUNG__: in Julia enumeration starts from `1`, not from `0`.  

Kernels are usually launched specifying the number of blocks to use and how many threads we need to use per block.  
Each thread must be mapped to work on a specific element. For example, if you have a vector, each thread has to be mapped on one vector element.  
Each thread has access to the size of its block via `blockDim.x`, to the index of its block within the grid via `blockIdx.x` and to its own index within its block via `threadIdx.x`. Using these variables, the formula 

`threadIdx.x + blockIdx.x * blockDim.x`

will map each thread to one element in the vector.



It may be the case that an execution configuration cannot be expressed that will create the exact number of threads needed for parallelizing a loop. Attempting to access non-existent elements can result in a runtime error.

A common example has to do with the desire to choose optimal block sizes. For example, due to GPU hardware traits, blocks that contain a number of threads that are a multiple of 32 are often desirable for performance benefits. Assuming that we wanted to launch blocks each containing 256 threads (a multiple of 32), and needed to run 1000 parallel tasks (a trivially small number for ease of explanation), then there is no number of blocks that would produce an exact total of 1000 threads in the grid, since there is no integer value 32 can be multiplied by to equal exactly 1000.

Here is an example of an idiomatic way to write an execution configuration when both N and the number of threads in a block are known, and an exact match between the number of threads in the grid and N cannot be guaranteed. It ensures that there are always at least as many threads as needed for N, and only 1 additional block's worth of threads extra, at most:

```cpp
// Assume `N` is known
int N = 100000;

// Assume we have a desire to set `threads_per_block` exactly to `256`
size_t threads_per_block = 256;

// Ensure there are at least `N` threads in the grid, but only 1 block's worth extra
size_t number_of_blocks = (N + threads_per_block - 1) / threads_per_block;
```

Then the code must check that the dataIndex calculated by `threadIdx.x + blockIdx.x * blockDim.x` is less than `N`, the number of data elements. In C/C++ this should look like the following pseudo-code:

```cpp
int idx = threadIdx.x + blockIdx.x * blockDim.x;

  if (idx < N) // Check to make sure `idx` maps to some value within `N`
  {
    // Only do work if it does
  }
```



But often there are more data elements than there are threads in the grid. In such scenarios threads cannot work on only one element or else work is left undone. One way to address this programmatically is with a __grid-stride loop__. In a grid-stride loop, the thread’s first element is calculated as usual, with `threadIdx.x + blockIdx.x * blockDim.x`. The thread then strides forward by the number of threads in the grid `(blockDim.x * gridDim.x)`, in this case 8. It continues in this way until its data index is greater than the number of data elements


```cpp
  int indexWithinTheGrid = threadIdx.x + blockIdx.x * blockDim.x;
  int gridStride = gridDim.x * blockDim.x;

  for (int i = indexWithinTheGrid; i < N; i += gridStride)
  {
    // do work on a[i];
  }

```

## Porting part 2

In [29]:
function gpu_mult3!(a, b, c)
    index = index = (blockIdx().x - 1) * blockDim().x + threadIdx().x 
    stride = gridDim().x * blockDim().x
    #@cuprintln("thread $index, block $stride")
    for i = index:stride:length(c)
        @inbounds c[i] = a[i] * b[i]
    end
    return nothing
end

gpu_mult3! (generic function with 1 method)

In [28]:
c_d = CUDA.zeros(Float32, N);
@cuda threads=1024 gpu_mult3!(a_d, b_d, c_d)

CUDA.HostKernel for gpu_mult3!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

thread 801, block 1024
thread 802, block 1024
thread 803, block 1024
thread 804, block 1024
thread 805, block 1024
thread 806, block 1024
thread 807, block 1024
thread 808, block 1024
thread 809, block 1024
thread 810, block 1024
thread 811, block 1024
thread 812, block 1024
thread 813, block 1024
thread 814, block 1024
thread 815, block 1024
thread 816, block 1024
thread 817, block 1024
thread 818, block 1024
thread 819, block 1024
thread 820, block 1024
thread 821, block 1024
thread 822, block 1024
thread 823, block 1024
thread 824, block 1024
thread 825, block 1024
thread 826, block 1024
thread 827, block 1024
thread 828, block 1024
thread 829, block 1024
thread 830, block 1024
thread 831, block 1024
thread 832, block 1024
thread 929, block 1024
thread 930, block 1024
thread 931, block 1024
thread 932, block 1024
thread 933, block 1024
thread 934, block 1024
thread 935, block 1024
thread 936, block 1024
thread 937, block 1024
thread 938, block 1024
thread 939, block 1024
thread 940,

In [30]:
@test all(Array(c_d) .== 2.0f0)

Test Passed

In [31]:
function bench_gpu3!(a, b, c)
    CUDA.@sync begin
        @cuda threads=256 gpu_mult3!(a, b, c)
    end
end

bench_gpu3! (generic function with 1 method)

In [32]:
c_d = CUDA.zeros(Float32, N)
@btime bench_gpu3!(a_d, b_d, c_d);

  1.512 ms (70 allocations: 4.05 KiB)


In [38]:
function gpu_mult4!(a, b, c)
    index = index = (blockIdx().x - Int32(1)) * blockDim().x + threadIdx().x  #less registers!
    stride = gridDim().x * blockDim().x
    # @cuprintln("thread $index, block $stride")
    for i = index:stride:length(c)
        @inbounds c[i] = a[i] * b[i]
    end
    return nothing
end

gpu_mult4! (generic function with 1 method)

In [39]:
c_d = CUDA.zeros(Float32, N);
@cuda threads=1024 gpu_mult4!(a_d, b_d, c_d)

CUDA.HostKernel for gpu_mult4!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [40]:
@test all(Array(c_d) .== 2.0f0)

Test Passed

In [41]:
function bench_gpu4!(a, b, c)
    CUDA.@sync begin
        @cuda threads=256 gpu_mult4!(a, b, c)
    end
end

bench_gpu4! (generic function with 1 method)

In [42]:
c_d = CUDA.zeros(Float32, N);
@btime bench_gpu4!(a_d, b_d, c_d);

  1.512 ms (69 allocations: 4.02 KiB)


In [43]:
c_d = CUDA.zeros(Float32, N);
CUDA.registers(@cuda gpu_mult4!(a_d, b_d, c_d))

28

In [44]:
c_d = CUDA.zeros(Float32, N);
CUDA.registers(@cuda gpu_mult3!(a_d, b_d, c_d))

32

In the previous kernel in the for loop we iterated over `index:stride:length(y)`, this is a `StepRange`. Unfortunately, constructing a `StepRange` is slow as they can throw errors and they contain unnecessary computation when we just want to loop over them. Instead it is faster to use a `while` loop like so:

In [50]:
function gpu_mult5!(a, b, c)
    index = index = (blockIdx().x - Int32(1)) * blockDim().x + threadIdx().x 
    stride = gridDim().x * blockDim().x
    # @cuprintln("thread $index, block $stride")
    
    i = index
    while i <= length(c)
        @inbounds c[i] = a[i] * b[i]
        i += stride
    end
    return nothing
end

gpu_mult5! (generic function with 1 method)

In [51]:
c_d = CUDA.zeros(Float32, N);
@cuda threads=1024 gpu_mult5!(a_d, b_d, c_d)

CUDA.HostKernel for gpu_mult5!(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [52]:
@test all(Array(c_d) .== 2.0f0)

Test Passed

In [53]:
function bench_gpu5!(a, b, c)
    CUDA.@sync begin
        @cuda threads=256 gpu_mult5!(a, b, c)
    end
end

bench_gpu5! (generic function with 1 method)

In [54]:
c_d = CUDA.zeros(Float32, N);
@btime bench_gpu5!(a_d, b_d, c_d);

  1.522 ms (73 allocations: 4.95 KiB)


In [55]:
c_d = CUDA.zeros(Float32, N);
CUDA.registers(@cuda gpu_mult5!(a_d, b_d, c_d))

18

## Atomics
`CUDA.jl` provides a low-level `atomic_` functions mapping directly on hardware instructions

In [56]:
function atomic_kernel(a)
    CUDA.atomic_add!(pointer(a), Int32(1))
    return
end

atomic_kernel (generic function with 1 method)

In [57]:
a = cu(Int32[1])

1-element CuArray{Int32, 1, CUDA.Mem.DeviceBuffer}:
 1

In [58]:
@cuda atomic_kernel(a)

CUDA.HostKernel for atomic_kernel(CuDeviceVector{Int32, 1})

In [59]:
a

1-element CuArray{Int32, 1, CUDA.Mem.DeviceBuffer}:
 2

## Dynamic parallelism  
Where kernels are normally launched from the host, using dynamic parallelism it is also possible to launch kernels from within a kernel. This is useful for recursive algorithms, or for algorithms that otherwise need to dynamically spawn new work.

Device-side launches are also done using the `@cuda` macro, but require setting the `dynamic` keyword argument to true:

In [60]:
function outer()
    @cuprint("Hello ")
    @cuda dynamic=true inner()
    return
end

outer (generic function with 1 method)

In [61]:
function inner()
    @cuprintln("World!")
    return
end

inner (generic function with 1 method)

In [62]:
@cuda outer()

CUDA.HostKernel for outer()

Hello World!


Within a kernel, only a very limited subset of the CUDA API is available:  
- synchronization: `device_synchronize`
- streams: `CuDeviceStream` constructor, `unsafe_destroy!` destuctor

# __CHECKPOINT__

## __Q1__
The following function estimates π with n samples. 
```julia
function estimatepi(n)
    area_circle = 0
    @inbounds for i = 1:n
        x, y = rand(), rand();
        r = x^2 + y^2
        if r < 1.0
            area_circle +=1
        end
    end
    return 4* area_circle/n
end
```  
- Parallelize the code using ```julia CUDA```
- Compare times and compute speedup for serial and GPU version of this code.

## __Q2__
Create three 2D arrays a, b, and c 10x10: 
- fill them with some initial value and then move the data to device 
- broadcast and sum them together in both host and device
- compare times for host and device operations.

## __Q3__
Consider the following kernel adding two vectors:

```julia
using CUDA, BenchmarkTools

function gpu_add!(y, x)
    for i = 1:length(c)
        @inbounds y[i] += x[i]
    end
    return
end
```

- Try to port this function to GPU by following what has been done for the example made on vector multiplication using as input vectors `x_d = CUDA.fill(1.0f0, 2^28)` and `y_d = CUDA.fill(2.0f0, 2^28)`
- Benchmark your fastest implementation against the slowest one and compute the speedup
- For every implementation compute the amount of registers used by using `CUDA.registers(@cuda gpu_add!(y_d, x_d))`
- profile your code with `CUDA.@profile`

## __Q1 solution__

In [ ]:
using CUDA, GPUArrays, BenchmarkTools, InteractiveUtils, Test 

function find_pi_gpu(n)
 4 * sum(CuArray(rand(Float64, n)).^2 .+ CuArray(rand(Float64, n)).^2 .<= 1) / n
end

tGPU = @btime find_pi_gpu(10000000) 
tSerial = @btime estimatepi(10000000)

times = [tSerial, tGPU]
speedup = maximum(times) ./ times

In [ ]:
#alternative kernel

using CuArrays.CURAND
function pi_gpu(n)
    4 * sum(CUDA.rand(Float64, n).^2 .+ CUDA.rand(Float64, n).^2 .<= 1) / n
end

## __Q2 solution__

In [ ]:
using CUDA, GPUArrays, BenchmarkTools, InteractiveUtils, Test 

h_a = ones(10, 10)
h_b = ones(10, 10)
h_c = ones(10, 10)


@btime h_a .+ h_b .+ h_c

 # Device
d_a = cu(h_a)
d_b = cu(h_b)
d_c = cu(h_c)

@btime d_a .+ d_b .+ d_c;


## __Q3 solution__